# **Imports & Default**

In [ ]:
import os
import h5py
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch import nn
from torchvision import models
from tqdm.notebook import tqdm
from torchsummary import summary
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

In [ ]:
World = '/dicos_ui_home/acye/'
mpl.rcParams['figure.dpi'] = 400
mpl.rcParams['axes.grid'] = True
mpl.rcParams['axes.axisbelow'] = True
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['savefig.facecolor'] = 'white'
mpl.rcParams['axes.formatter.useoffset'] = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('PyTorch Version', torch.__version__, '\nCUDA:',torch.cuda.is_available())

# **Data**

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.FloatTensor(data.reshape([-1, 1, 8192]))
        self.targets = torch.LongTensor(target)
    
    def __len__(self):
        
        return len(self.data)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        return x, y

In [ ]:
n = 50000
with h5py.File(World+'Data/CCSN/CCSN_Pan_SNR_Free.h5', 'r') as hdf:
    Strain = np.r_[hdf['Model_NR']['Hp'][:n], hdf['Model_SR']['Hp'][:n], hdf['Model_FR']['Hp'][:n], hdf['Noise']['Strain'][:3*n]]

n_v = int(n/8)
with h5py.File(World+'Data/CCSN/CCSN_Pan_SNR_Free_Test_Validation.h5', 'r') as hdf_v:
    Strain_valid = np.r_[hdf_v['Model_NR']['Hp'][:n_v], hdf_v['Model_SR']['Hp'][:n_v], hdf_v['Model_FR']['Hp'][:n_v], hdf_v['Noise']['Strain'][:3*n_v]]

In [ ]:
target = np.r_[np.c_[np.ones(n*3), np.zeros(n*3)], 
               np.c_[np.zeros(n*3), np.ones(n*3)]]

target_valid = np.r_[np.c_[np.ones(n_v*3), np.zeros(n_v*3)], 
                     np.c_[np.zeros(n_v*3), np.ones(n_v*3)]]

In [ ]:
dataset = MyDataset(Strain, target)
dataloader = DataLoader(dataset, batch_size=125, shuffle=True)

dataset_valid = MyDataset(Strain_valid, target_valid)
dataloader_valid = DataLoader(dataset_valid, batch_size=125, shuffle=True)

# **Model**

In [ ]:
class Torch_AE(nn.Module):
    def __init__(self):
        super(Torch_AE, self).__init__()
        self.Conv1 = nn.Conv1d(1, out_channels = 120, kernel_size = 3, stride = 1, padding = 1)
        self.Conv2 = nn.Conv1d(120, out_channels = 80, kernel_size = 3, stride = 1, padding = 1)
        self.Conv3 = nn.Conv1d(80, out_channels = 40, kernel_size = 3, stride = 1, padding = 1)
        self.Conv4 = nn.Conv1d(40, out_channels = 40, kernel_size = 3, stride = 1, padding = 1)
        self.L1 = nn.Linear(20480, 200)
        self.L2 = nn.Linear(200, 100)
        self.L3 = nn.Linear(100, 2)
        
    def forward(self, x):
        x = F.dropout(nn.MaxPool1d(kernel_size=2)(F.relu(self.Conv1(x))), p = 0)
        x = F.dropout(nn.MaxPool1d(kernel_size=2)(F.relu(self.Conv2(x))), p = 0)
        x = F.dropout(nn.MaxPool1d(kernel_size=2)(F.relu(self.Conv3(x))), p = 0)
        x = F.dropout(nn.MaxPool1d(kernel_size=2)(F.relu(self.Conv4(x))), p = 0)
        x = torch.flatten(x, 1)
        x = F.sigmoid(self.L1(x))
        x = F.sigmoid(self.L2(x))
        x = F.softmax(self.L3(x), dim = 1)
        return x

# **Training**

In [ ]:
epochs = 10
learning_rate = 0.001

model = Torch_AE().to(device)

criterion = nn.CrossEntropyLoss()


opt = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 1e-5)


cost_value = np.array([])
cost_valid_value = np.array([])

In [ ]:
for i in range(epochs):
    for j, (x, y) in enumerate(tqdm(dataloader, desc=f'Epoch {i+1}')):
        x = x.to(device)
        y = y.to(device)

        p_value = model(x)
        cost = criterion(p_value, torch.argmax(y, dim = 1))
        
        opt.zero_grad()
        cost.backward()
        opt.step()
        
    print('========')
    print(f'Cost{round(cost.item(), 2)}')
    print('========', '\n')

    with torch.no_grad():
        v_cost = 0
        for run, (a, b) in enumerate(dataloader_valid):
            a = a.to(device)
            b = b.to(device)
            
            p_value = model(a)
            cost_valid = criterion(p_value, torch.argmax(b, dim = 1))
            
            v_cost += cost_valid.item()
            
        cost_valid_value = np.append(cost_valid_value, v_cost/(run+1))
        
        print('===================')
        print(f'Validation Cost{round(cost_valid.item(), 2)}')
        print('===================', '\n')
    
    cost_value = np.append(cost_value, cost.item())
print('Train Ends!')

# **Plots**

In [ ]:
total_epoch = np.linspace(1, epochs, epochs)
plt.title('Loss value to epoch')
plt.plot(total_epoch, cost_value, label = 'Training Set')
plt.plot(total_epoch, cost_valid_value, label = 'Validation Set')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.savefig('Free SNR Sigmoid.png')
plt.legend()
plt.show()

In [ ]:
t_metrix = np.zeros([2,2])
metrix = np.zeros([2,2])
with torch.no_grad():
    for num, (x, y) in enumerate(tqdm(dataloader)):
        Real_label = y.numpy()
        Prediction = model(x.to(device)).cpu().detach().numpy()
        
        for i in range(2): 
            for j in range(2):
                if np.sum(Real_label[:,j]) != 0:
                    metrix[i, j] =  np.dot(Real_label[:,j], 
                                           Prediction[:,i]
                                          )/np.sum(Real_label[:,j])
                if np.sum(Real_label[:,j]) == 0 :
                    metrix[i, j] =  np.dot(Real_label[:,j], 
                                           Prediction[:,i]
                                          )/(np.sum(Real_label[:,j])+1e-20)
        t_metrix += metrix
    metrix = t_metrix/(num+1)
    TPR = metrix[0,0]
    FAR = metrix[0,1]/(metrix[0,0] + metrix[0,1])
    
print(f'TPR: {TPR}')
print(f'FAR: {FAR}')

In [ ]:
lab = np.array([['True P', 'False P'], ['False N', 'True N']])
color = plt.pcolormesh([metrix[1], metrix[0]], cmap = 'Wistia', vmin=0, vmax=1)
for i in range(2):
    for j in range(2):
        plt.text(i+.5, j+.5, f'{lab[1-j,i]}\n{round(metrix[1-j,i],3)}', ha = 'center', va = 'center')
plt.xticks([.5, 1.5], ['Injection', 'Noise'])
plt.yticks([.5, 1.5], ['Noise', 'Injection'], rotation=45)
plt.xlabel('Real Value')
plt.ylabel('Predict', rotation=0)
plt.title(f'Training set with:     TPR: {round(metrix[0, 0], 3)}FAR: {round(FAR, 3)}')
plt.colorbar(color)
plt.savefig('Training set performance Free SNR Sigmoid.png', bbox_inches = 'tight')
plt.show()

In [ ]:
t_metrix = np.zeros([2,2])
metrix = np.zeros([2,2])
with torch.no_grad():
    for num, (x, y) in enumerate(tqdm(dataloader_valid)):
        Real_label = y.numpy()
        Prediction = model(x.to(device)).cpu().detach().numpy()
        
        for i in range(2): 
            for j in range(2):
                if np.sum(Real_label[:,j]) != 0:
                    metrix[i, j] =  np.dot(Real_label[:,j], 
                                           Prediction[:,i]
                                          )/np.sum(Real_label[:,j])
                if np.sum(Real_label[:,j]) == 0 :
                    metrix[i, j] =  np.dot(Real_label[:,j], 
                                           Prediction[:,i]
                                          )/(np.sum(Real_label[:,j])+1e-20)
        t_metrix += metrix
    metrix = t_metrix/(num+1)
    TPR = metrix[0,0]
    FAR = metrix[0,1]/(metrix[0,0] + metrix[0,1])
    
print(f'TPR: {TPR}')
print(f'FAR: {FAR}')

In [ ]:
lab = np.array([['True P', 'False P'], ['False N', 'True N']])
color = plt.pcolormesh([metrix[1], metrix[0]], cmap = 'Wistia', vmin=0, vmax=1)
for i in range(2):
    for j in range(2):
        plt.text(i+.5, j+.5, f'{lab[1-j,i]}\n{round(metrix[1-j,i],3)}', ha = 'center', va = 'center')
plt.xticks([.5, 1.5], ['Injection', 'Noise'])
plt.yticks([.5, 1.5], ['Noise', 'Injection'], rotation=45)
plt.xlabel('Real Value')
plt.ylabel('Predict', rotation=0)
plt.title(f'Validation set with:   TPR: {round(metrix[0, 0], 3)} FAR: {round(FAR, 3)}')
plt.colorbar(color)
plt.savefig('Validation set performance Free SNR Sigmoid.png', bbox_inches = 'tight')
plt.show()

#  **Xavier**

In [ ]:
Mixed_model = model.to('cpu')
torch.save(Mixed_model, World+'CCSN/Saved_Model/Mixed_model_Sigmoid.pt')